# Spring Pendulum Analysis

This notebook examines the spring pendulum system, which is a pendulum with a variable-length spring instead of a rigid rod. 

## System Description

The spring pendulum is described by:
- $\theta$: angle from vertical
- $r$: length of pendulum (spring length)
- $p_{\theta}$: angular momentum
- $p_{r}$: radial momentum

The Hamiltonian is:
$$H(\theta, r, p_\theta, p_r) = \frac{p_r^2}{2m} + \frac{p_\theta^2}{2mr^2} - mgr\cos(\theta) + \frac{k}{2}(r-r_0)^2$$

where:
- $m$ is the mass
- $g$ is gravity
- $k$ is the spring constant
- $r_0$ is the equilibrium length of the spring

This system exhibits interesting dynamics with coupling between the oscillations in $r$ and $\theta$.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.integrate import solve_ivp
from scipy.integrate._ivp.rk import RK45, RK23, DOP853
import time

# Add the parent directory to sys.path to import invflow modules
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Import HamiltonianSystem for setting up the problem
from invflow.dynamics import HamiltonianSystem
from invflow.adaptive import create_projected_solver

# Define the spring pendulum system directly
# Variables: theta (angle), r (radius), p_theta (angular momentum), p_r (radial momentum)
r, theta, p_r, p_theta = sp.symbols('r theta p_r p_theta', real=True)
variables = [r, theta, p_r, p_theta]

# Parameters
m = 1  # mass
g = 1  # gravity
k = 1  # spring constant

# Hamiltonian for spring pendulum
r_0 = 1.0  # equilibrium length of spring
T = p_r**2/(2*m) + p_theta**2/(2*m*r**2)  
V = -m*g*r*sp.cos(theta) + k*(r-r_0)**2/2
H_expr = T + V

# Create the Hamiltonian system
hamiltonian_system = HamiltonianSystem(H_expr, variables)
f_pendulum = hamiltonian_system.f  # Dynamics function
invariants = [H_expr]  # The Hamiltonian is conserved

# Create a function to compute energy
H_fun = hamiltonian_system.H

print("Hamiltonian:", H_expr)
print("Variables:", variables)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Hamiltonian: p_r**2/2 + p_theta**2/(2*r**2) - r*cos(theta) + (r - 1.0)**2/2
Variables: [r, theta, p_r, p_theta]


In [ ]:
r, theta, p_r, p_theta, t, k = sp.symbols('r theta p_r p_theta t k', real=True)
variables = [r, theta, p_r, p_theta]

def phi(x, m=1.0, r0=1.0):
    r, theta, pr, pth = x
    y1 = r
    y2 = -r*sp.cos(theta) + (r - r0)**2/2
    y3 = pr/sp.sqrt(2*m)
    y4 = pth/(sp.sqrt(2*m)*r)
    return sp.Matrix([y1, y2, y3, y4])

def phi_inv(y, m=1.0, r0=1.0, x_prev=None):
    y1, y2, y3, y4 = y
    r = y1

    # cos(theta) from y
    arg = (-y2 + 1/2*(r - r0)**2) / r

    prev_theta = None
    if x_prev is not None and len(x_prev) >= 2 and np.isfinite(x_prev[1]):
        prev_theta = float(x_prev[1])

    theta = sp.acos(arg)

    # momenta
    pr  = sp.sqrt(2*m) * y3
    pth = sp.sqrt(2*m) * r * y4

    return sp.Matrix([r, theta, pr, pth])

def G(y):
    r, theta, pr, pth = y
    return theta + pr**2 + pth**2

# H_expr
exp = sp.Matrix([[sp.exp(0), 0,0,0], [0,sp.exp(2*t),0,0], [0,0,sp.exp(t),0], [0,0,0,sp.exp(t)]])
k = 2
# sp.factor(G(phi_inv(exp*phi(variables))))

H = lambda x: H_expr.subs({r: x[0], theta: x[1], p_r: x[2], p_theta: x[3]})
print('conjugate invariant: ', G(phi(variables))-H(variables))
print('homogeneous in conjugate space: ', sp.expand(G(exp*phi(variables)) - sp.exp(k*t)*G(phi(variables))))
print('c-homogeneous in physical space: ', sp.expand(H(phi_inv(exp*phi(variables)))-sp.exp(k*t)*H(variables)))


conjugate invariant:  -1.11022302462516e-16*p_r**2 - 1.11022302462516e-16*p_theta**2/r**2
homogeneous in conjugate space:  0
c-homogeneous in physical space:  0
